In [6]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from mne.datasets import sample
from mne.decoding import cross_val_multiscore, LinearModel, GeneralizingEstimator, Scaler, \
                         Vectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, StratifiedShuffleSplit, \
                                    RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import argparse

# Set args

In [7]:
class arguments():
    SAVE_EPOCH_ROOT = '../../data/preprocessed/epochs/aft_ICA_rej/'
    SAVE_RESULT_ROOT = '../../results/decoding/permtest/'
    cond_filter ='none' # {none,non_symm}
    cond_block ='early' #{early,later}
    cond_time = 'prestim' #{prestim,poststim}
    cond_decoding = 'none' #{none,removeevoked,resampled}
    subj_num = 1
    applyBaseline_bool = 0
    pre_tmin = -0.4
    pre_tmax = 0.05
    post_tmin = 0.05
    post_tmax = 0.45
    num_classes = 2
    normalization_type = 'normal'# {normal,lstmPaper}
    gen_rand_perm = 0
    null_max_iter = 10000
    loop_null_iter = 5
    gen_decoder_scores = 1
    n_splits = 5
    random_state = 42 
    max_iter = 10000
    n_jobs = 1
    scoring = 'roc_auc'
    
args = arguments()

# Read each subj and preprocess data

In [3]:
"""
Reading and preparing epoch data to create each 4 grous and 2 pattern
"""
def read_prep_epochs(args):
    if args.applyBaseline_bool:
        filename_epoch = args.SAVE_EPOCH_ROOT + \
                         'epochs_sec_applyBaseline_subj%s-afterRejICA-epo.fif' \
                          %args.subj_num
    else:
        filename_epoch = args.SAVE_EPOCH_ROOT + \
                         'epochs_sec_subj%s-afterRejICA-epo.fif' \
                         %args.subj_num
    epochs_orig = mne.read_epochs(filename_epoch, proj=True, preload=True,
                                  verbose=None)
    epochs = epochs_orig.copy()
    subset = epochs['pred']['non'].copy()
    subset = subset.pick_types(eeg=True)
    if (args.cond_decoding=='removeevoked'):
        # REMOVE EVOKED RESP.
        subset.subtract_evoked()    # remove evoked response
    elif (args.cond_decoding=='resampled'):
        # RESAMPLE
        subset = subset.resample(args.n_resampling, npad='auto')
    else:
        pass
    ##==========================================================================
    if subset['Block==7'].metadata.Ptrn_Type.values.shape[0]>0:
       main_ptrn = subset['Block==7'].metadata.Ptrn_Type.values[0]
    else:
       main_ptrn = subset['Block==8'].metadata.Ptrn_Type.values[0]
    ##==========================================================================
    if args.cond_block=='early': #block 3-6
        subset = subset['Block<7'].copy()
        subset = subset['Block>2'].copy()
    elif args.cond_block=='later':#block 7-10
        subset = subset['Block<11'].copy()
        subset = subset['Block>6'].copy()
    ##==========================================================================
    if (args.cond_time=='prestim'):
        subset= subset.crop(tmin=-0.4, tmax=0.05)
    if (args.cond_time=='poststim'):
        subset= subset.crop(tmin=0.05, tmax=0.45)
    ##==========================================================================
    # Group data based on the previous trial
    Grp1 = subset['Trgt_Loc_prev==1'].copy()
    Grp2 = subset['Trgt_Loc_prev==2'].copy()
    Grp3 = subset['Trgt_Loc_prev==3'].copy()
    Grp4 = subset['Trgt_Loc_prev==4'].copy()
    if main_ptrn==1:
        Grp1 = Grp1['Trgt_Loc_main!=4'].copy()
        Grp2 = Grp2['Trgt_Loc_main!=1'].copy()
        Grp3 = Grp3['Trgt_Loc_main!=2'].copy()
        Grp4 = Grp4['Trgt_Loc_main!=3'].copy()
    ##==========================================================================
    frequencies = np.arange(3, 13, 2)
    if args.cond_decoding=='non_symm':
        Grp1 = apply_nonSymm_filter(Grp1, frequencies)
        Grp2 = apply_nonSymm_filter(Grp2, frequencies)
        Grp3 = apply_nonSymm_filter(Grp3, frequencies)
        Grp4 = apply_nonSymm_filter(Grp4, frequencies)
    ##==========================================================================
    print('the pattern for this subj is :=====================================')
    print(main_ptrn)
    print('          ')
    print('===================================================================')
    ##==========================================================================
    # Normalizing the data for each subject
    if args.normalization_type=='normal':
        Grp1._data = (Grp1._data - np.mean(Grp1._data)) / np.std(Grp1._data)
        Grp2._data = (Grp2._data - np.mean(Grp2._data)) / np.std(Grp2._data)
        Grp3._data = (Grp3._data - np.mean(Grp3._data)) / np.std(Grp3._data)
        Grp4._data = (Grp4._data - np.mean(Grp4._data)) / np.std(Grp4._data)
    elif args.normalization_type=='lstmPaper':
        Grp1._data = (2 * (Grp1._data - np.min(Grp1._data))) \
                        / (np.max(Grp1._data) - np.min(Grp1._data) - 1)
        Grp2._data = (2 * (Grp2._data - np.min(Grp2._data))) \
                        / (np.max(Grp2._data) - np.min(Grp2._data) - 1)
        Grp3._data = (2 * (Grp3._data - np.min(Grp3._data))) \
                        / (np.max(Grp3._data) - np.min(Grp3._data) - 1)
        Grp4._data = (2 * (Grp4._data - np.min(Grp4._data))) \
                        / (np.max(Grp4._data) - np.min(Grp4._data) - 1)
    ##==========================================================================
    return Grp1, Grp2, Grp3, Grp4, main_ptrn

# Set up a decoder and apply temporal generalization

In [4]:
"""
Temporal Generalization
"""
def apply_tempGen(args, Grp_data, cv):
    le = LabelEncoder()
    clf_SVC  = make_pipeline(
                          StandardScaler(),
                          LinearModel(LinearSVC(random_state=args.random_state,
                                                max_iter=args.max_iter)))
    X=Grp_data.copy()._data
    y=le.fit_transform(Grp_data.copy().metadata.Trgt_Loc_main)

    time_gen = GeneralizingEstimator(clf_SVC, scoring=args.scoring,
                                     n_jobs=args.n_jobs, verbose=True)
    print(np.unique(y))
    print(np.unique(Grp_data.copy().metadata.Trgt_Loc_main))

    scores = cross_val_multiscore(time_gen, X, y, cv=cv, n_jobs=args.n_jobs)
    scores = np.mean(scores, axis=0) #scores with cv
    scores_diag = np.diag(scores)
    scores_substract = scores_diag - 0.5
    scores_pck = (scores.copy(), scores_diag.copy(), scores_substract.copy())

    # Without using cv, train and test on the same data
    X = Grp_data.copy()._data
    y = le.fit_transform(Grp_data.copy().metadata.Trgt_Loc_main)
    time_gen.fit(X=X ,y=y)
    scores = time_gen.score(X=X, y=y) #scores without cv
    scores_diag = np.diag(scores)
    scores_pck_fit = (scores.copy(), scores_diag.copy())

    return scores_pck, scores_pck_fit


# Plot functions

In [117]:
def smooth(y, window, mode):
    box = np.ones(window)/window
    y_smooth = np.convolve(y, box, mode=mode)
    return y_smooth

def plot_scores(scores):
    fig, ax = plt.subplots(1, 1)
    plt.tight_layout()
    im = ax.imshow(scores, interpolation='lanczos', origin='lower', cmap='RdBu_r',
                   extent=subset.times[[0, -1, 0 , -1]], vmin=0., vmax=1.)
    ax.set_xlabel('Testing Time (s)')
    ax.set_ylabel('Training Time (s)')
    ax.set_title('Temporal generalization')
    ax.axvline(0, color='k')
    ax.axhline(0, color='k')
    ax.xaxis.set_ticks_position('bottom')
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    plt.show()

def plot_scores_diag(scores_diag, apply_smooth):
    if apply_smooth:
        window=50
        mode='valid'
        scores_diag = smooth(y, window, mode)
        print(subset.times.shape)
        print(y_smooth.shape)
    fig, ax = plt.subplots()
    ax.plot(subset.times, scores_diag, label='score')
    ax.axhline(.5, color='k', linestyle='--', label='chance')
    ax.set_xlabel('Times')
    ax.set_ylabel('AUC')  # Area Under the Curve
    ax.legend()
    ax.axvline(.0, color='k', linestyle='-')
    ax.set_title('Sensor space decoding')
    plt.tight_layout()
    plt.show()

# Statistical analysis

## Plot functions

In [121]:
def do_time_bin(data, indx, sbt):
    if sbt==0:
        avgs=np.zeros(len(indx))
        bs=np.array(np.split(data, indx))
        for ii in range(len(indx)):
            avgs[ii]=bs[ii].mean()
    if sbt==1:
         avgs=np.zeros([data.shape[0],len(indx)])
         aa=np.zeros(len(indx))
         for jj in range(data.shape[0]):
             bs=np.array(np.split(data[jj,:], indx))
             for ii in range(len(indx)):
                 aa[ii]=bs[ii].mean()
             avgs[jj,:]=aa
    if sbt==2:
         avgs=np.zeros([len(indx),len(indx)])
         aa=np.zeros(len(indx))
         for jj in range(data.shape[0]):
             bs1=np.array(np.split(data[jj,:], indx))
             bs2=np.array(np.split(data[:,jj], indx))
             for ii in range(len(indx)):
                 avgs[ii,:]=bs1[ii].mean()
                 avgs[:,ii]=bs2[ii].mean()

    return avgs

In [161]:
def set_fonts():
    from matplotlib.font_manager import FontProperties

    font = FontProperties()
    font.set_family('serif')
    font.set_name('Calibri')
    return font

def plot_scores_stat(diag_scores, clusts):
    font=set_fonts();
    [t_obs, clusters, clusters_pv, H0] = clusts
    # binned times
    times=np.asarray([-0.4,-0.3,-0.2,-0.1,0.1,0.2,0.3,0.4])
    extent_times=subset.times[[0, -1, 0, -1]]
    
    # Plot the diagonal (it's exactly the same as the time-by-time decoding above)
    fig, ax = plt.subplots()
    plt.tight_layout()
    ax.plot(times, diag_scores, label='score')
    ax.axhline(.5, color='k', linestyle='--', label='chance')
    plt.ylim([0.43,0.65])
    ax.axvline(.0, color='k', linestyle='-')

    for i_clu, clu_idx in enumerate(clusters):
        clu_idx=clu_idx[0]
        print(clu_idx)
        # unpack cluster information, get unique indices
        if clusters_pv[i_clu] <= 0.05:
            h = plt.axvspan(times[clu_idx[0]], times[clu_idx[-1] - 1],
                            color='r', alpha=0.3)
            plt.legend((h, ), ('cluster p-value < 0.05', ))
        else:
            plt.axvspan(times[clu_idx[0]], times[clu_idx[-1] - 1], color=(0.3, 0.3, 0.3),
                        alpha=0.3)

    plt.tight_layout()
    plt.xlabel('Times',  fontproperties=font, fontsize=12, fontweight='bold')
    plt.ylabel('AUC', fontproperties=font, fontsize=12, fontweight='bold')#, labelpad=16,)
    plt.title('Decoding over time', fontproperties=font, fontweight='bold', fontsize=16)

    plt.legend(fontsize=11)
    plt.tight_layout()

## Statistial analysis

In [172]:
def stat_anal(scores_pck):
    indx=[26,51,76,101,126,151,176,201]
    score, score_diag, score_subtract = scores_pck
    binned_score = do_time_bin(score, indx, 2)
    binned_score_diag = do_time_bin(score_diag, indx, 0)
    binned_score_subtract = do_time_bin(score_subtract, indx, 0)
    print(score_subtract.shape)
    score_subtract=score_subtract[:, np.newaxis, np.newaxis] # [:,:, np.newaxis] when added more subjects
    print(score_subtract.shape)
    t_obs, clusters, clusters_pv, H0 = mne.stats.spatio_temporal_cluster_1samp_test(score_subtract, tail=0)
    clust_pck = [t_obs, clusters, clusters_pv, H0]
    
    return binned_score_diag, clust_pck

### Stat analysis for stratified shuffle split result (Alex suggestion)

In [ ]:
binned_score_diag_sss, clust_pck_sss = stat_anal(scores_sss)

In [ ]:
plot_scores_stat(binned_score_diag_sss, clust_pck_sss)

# Select some subjects

In [42]:
# subj_indx = [ 1,  2,  3,  4,  5,  7,  8,  9, 10, 12, 15, 16, 17, 18, 19, 20, 21,
#        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
#        39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 55, 56, 57, 58, 59,
#        60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]

# selected_subj = [subj_indx[0], subj_indx[29], subj_indx[30], subj_indx[35],\
#                  subj_indx[49], subj_indx[60], 25, 15, 21, 48]

# print(selected_subj)

selected_subj = [1, 34]

In [49]:
cv = StratifiedShuffleSplit(n_splits=args.n_splits, random_state=args.random_state)                                                    

In [39]:
avgp1_score=[]
avgp2_score=[]

avgp1_diag=[]
avgp2_diag=[]

avgp1_substract=[]
avgp2_substract=[]


for subj_id in selected_subj:
    args.subj_num = subj_id
    Grp1, Grp2, Grp3, Grp4, main_ptrn = read_prep_epochs(args)
    
    sc_pck_G1, sc_pck_fit_G1 = apply_tempGen(args, Grp_data, cv)
    sc_pck_G2, sc_pck_fit_G2 = apply_tempGen(args, Grp_data, cv)
    sc_pck_G3, sc_pck_fit_G3 = apply_tempGen(args, Grp_data, cv)
    sc_pck_G4, sc_pck_fit_G4 = apply_tempGen(args, Grp_data, cv)
    
    # unpack them
    score_G1, score_diag_G1, score_subtract_G1 = sc_pck_G1
    score_G2, score_diag_G2, score_subtract_G2 = sc_pck_G2
    score_G3, score_diag_G3, score_subtract_G3 = sc_pck_G3
    score_G4, score_diag_G4, score_subtract_G4 = sc_pck_G4
    
    # integrate them for all subjects
    if main_ptrn == 1:
        avgp1_score.append(sc_pck_G1)
        avgp1_diag.append(score_diag_G1)
        avgp1_substract.append(score_subtract_G1)
        
        avgp1_score.append(sc_pck_G2)
        avgp1_diag.append(score_diag_G2)
        avgp1_substract.append(score_subtract_G2)
        
        avgp1_score.append(sc_pck_G3)
        avgp1_diag.append(score_diag_G3)
        avgp1_substract.append(score_subtract_G3)
        
        avgp1_score.append(sc_pck_G4)
        avgp1_diag.append(score_diag_G4)
        avgp1_substract.append(score_subtract_G4)
    elif man_ptrn == 2:
        avgp2_score.append(sc_pck_G1)
        avgp2_diag.append(score_diag_G1)
        avgp2_substract.append(score_subtract_G1)
        
        avgp2_score.append(sc_pck_G2)
        avgp2_diag.append(score_diag_G2)
        avgp2_substract.append(score_subtract_G2)
        
        avgp2_score.append(sc_pck_G3)
        avgp2_diag.append(score_diag_G3)
        avgp2_substract.append(score_subtract_G3)
        
        avgp2_score.append(sc_pck_G4)
        avgp2_diag.append(score_diag_G4)
        avgp2_substract.append(score_subtract_G4)
            

Reading ../../data/preprocessed/epochs/aft_ICA_rej/epochs_sec_subj1-afterRejICA-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 129) active
    Found the data of interest:
        t =    -400.00 ...    5000.00 ms
        0 CTF compensation matrices available
915 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)


KeyboardInterrupt: 

# General decoder params

In [34]:
nSplits=5
scoring='roc_auc'
randState=0
maxIter=10000

# Set up a decoder


## Using StandardScaler from sklearn.preprocessing:
### mne.decoding.Scaler
    scales each channel 
    using mean and standard deviation computed across all of its time points and epochs. 
###  sklearn.preprocessing.StandardScaler offered by scikit-learn
    These scale each classification feature, e.g. each time point for each channel, 
    with mean and standard deviation computed across epochs